In [1]:
import os
import glob

import tensorflow as tf
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, MaxPooling2D, Dropout, Lambda
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers

from sklearn.model_selection import train_test_split

from PIL import Image                                                                                                                               
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
import h5py


%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\Xyrion\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Training images path
PATH="images_background/"

#Validation and test images path
PATH_TEST = "images_evaluation/"

In [3]:
alph_type = np.array(os.listdir(PATH)) #Give the different types of alphabet in our training data
alph_num_char ={alphabet:len(os.listdir(f'{PATH}{alphabet}')) for alphabet in alph_type}
num_of_char = alph_num_char.values()
total_char = sum(num_of_char)

alph_num_char_ex={}
for alphabet in alph_type:
    char_list=os.listdir(f'{PATH}{alphabet}')
    for char in char_list:
        alph_num_char_ex[(alphabet,char)]= len(os.listdir(f'{PATH}{alphabet}/{char}'))

num_of_example = alph_num_char_ex.values()
total_example = sum(num_of_example) 

In [4]:
Y_train = np.array([i//20+1 for i in range(total_example)])
Y_train=Y_train.reshape(*Y_train.shape,1)

In [5]:
#Train set
imagePath = glob.glob(f"{PATH}*/*/*.png")

#Test set
testPath = glob.glob(f"{PATH_TEST}*/*/*.png")

In [6]:
SZ=60 #Dimension of the output image expected

In [7]:
X_train = np.array([resize(mpimg.imread(i), (SZ,SZ), mode='reflect') for i in imagePath] )
channel_sz = 1 #number of channel
X_train= X_train.reshape(*X_train.shape, channel_sz)

In [8]:
#Test set
test_array = np.array([resize(mpimg.imread(i), (SZ,SZ), mode='reflect') for i in testPath] )
test_array= test_array.reshape(*test_array.shape, channel_sz)
test_array.shape

class_char_test = np.array([i//20+1 for i in range(test_array.shape[0])])
class_char_test = class_char_test.reshape(*class_char_test.shape,1)
class_char_test.shape

(13180, 1)

In [9]:
X_val, X_test, Y_val, Y_test = train_test_split(test_array, class_char_test, test_size=0.7, stratify= class_char_test)
X_val.shape, X_test.shape, Y_val.shape, Y_test.shape

((3954, 60, 60, 1), (9226, 60, 60, 1), (3954, 1), (9226, 1))

In [10]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 64)
            positive -- the encodings for the positive images, of shape (None, 64)
            negative -- the encodings for the negative images, of shape (None, 64)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[:,0:64], y_pred[:,64:128], y_pred[:,128:196]
    
    #Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    #Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    #Subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = tf.to_float(tf.greater(basic_loss,0))
    num_positive_triplets = tf.reduce_sum(valid_triplets)
    #Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))/ alpha * 100 #/ (num_positive_triplets + 1e-16)
    
    return loss

In [11]:
def triplet_acc(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 64)
            positive -- the encodings for the positive images, of shape (None, 64)
            negative -- the encodings for the negative images, of shape (None, 64)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[:,0:64], y_pred[:,64:128], y_pred[:,128:196]
    
    #Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    #Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    #Subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Count number of positive triplets (where triplet_loss > 0)
    hard_triplets = tf.to_float(tf.greater(basic_loss,alpha))
    num_hard_triplets = tf.reduce_sum(hard_triplets)
    #Count number of triplets
    all_triplets = tf.reduce_sum(tf.to_float(tf.greater(basic_loss,-10**10)))
    
    #Accuracy
    acc = 1 - num_hard_triplets/all_triplets
    
    return acc

In [12]:
def create_triplets(X, Y, num=1):
    """
    Create a list of valid triplets
    
    Arguments:
    X -- array of images
    Y -- array of classes corresponding to each image
    num -- number of negative images for each valid anchor and positive images - must be positive
           if num = 0, all possible valid couples are created
            For example : for one valid (A,P) couple we can select 'num' random N images. 
                          Thus 'num' triplets are created for this (A,P) couple

    
    Returns:
    (A,P,N) -- python tuple containing 3 arrays : 
            A -- the array for the anchor images, of shape (None, 64)
            P -- the array for the positive images, of shape (None, 64)
            N -- the array for the negative images, of shape (None, 64)
    """

    Y = Y.reshape(Y.shape[0],)
    A = []
    P = []
    N = []
    
    #We loop over all possible valid (A,P)
    for i in range(X.shape[0]):  
        list_pos = X[Y==Y[i]]
        for j in list_pos:
            #We provide a number 'num' of triplets for each valid (A,P)
            if num >=1:
                for k in range(num):
                    rand_num = np.random.randint(X.shape[0])
                    if np.array_equal(X[i],j) == False:
                        A.append(X[i])
                        P.append(j)
                        while np.array_equal(Y[rand_num], Y[i]):
                            rand_num = np.random.randint(X.shape[0])
                        N.append(X[rand_num])
            if num == 0:
                for k in range(X.shape[0]):
                    if np.array_equal(X[i],j) == False:
                        if np.array_equal(Y[i],Y[k]) == False:
                            A.append(X[i])
                            P.append(j)
                            N.append(X[k])
    
    A = np.array(A)
    P = np.array(P)
    N = np.array(N)
    
    return (A, P, N)

In [13]:
def pairwise_distances(embeddings, squared=False):
    """ 
    Compute the 2D matrix of distances between all the embeddings.
    
    Arguments:
        embeddings : tensor of shape (batch_size, embed_dim)
        squared : Boolean. If true, output the pairwise squared euclidean distance 
                matrix instead of the euclidean distance matrix
                
    Returns:
        pairwise_distances : tensor of shape(batch_size,batch_size)
    
    """
    
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = tf.matmul(embeddings, embeddings.T)
    
    # Get the squared L2 norm for each embeddings (correspond)
    square_norm = tf.diag_part(dot_product)
    
    # Compute the pairwise distance matrix :
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm,1) - 2.0 * dot_product + tf.expand_dims(square_norm, 0)
    
    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.maximum(distances, 0.0)
    
    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16
        
        distances = tf.sqrt(distances)
        
        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)
    return distances

In [ ]:
class TraningData(Sequence):
    def __init__(self, temp_mapping):
        self.temp_mapping = temp_mapping ###to update
        self.class_to_list_files=defaultdict(list) ### to update
        self.list_all_files = list(temp_mapping.keys())
        self.range_files = list(range(len(self.list_all_files)))
        
    def get_sample(self):
        random_image = 
        


In [18]:
a=np.random.choice([range(3)],1)

ValueError: a must be 1-dimensional

In [20]:
list(range(3))

[0, 1, 2]

In [ ]:
triplets_list_val = create_triplets(X_val, Y_val)
print([i.shape for i in triplets_list_val])